In [1]:
# sc.stop()

NameError: name 'sc' is not defined

In [2]:
from pyspark import SparkContext, SparkConf

if not 'sc' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')  # Spark will use all cores (*) available
    sc = SparkContext(conf=conf) # Initialize SparkContext sc with the above configuration conf 

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

22/10/19 12:20:38 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.26.54.250 instead (on interface eth0)
22/10/19 12:20:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 12:20:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Create SparkDataframe using Schema and Data . Also print the schema and dataframe

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() # getOrCreate() will return an existing SparkSession if there is one, or create a new one if there is none.


# define schema using DDl
schema = "id INT, name STRING, age INT"

data= [(1, "John", 19), (2, "Smith", 29), (3, "Adam", 35), (4, "Henry", 50), (5, "Mike", 78), (6, "Mary", 19), (7, "Peter", 29), (8, "Cora", 35), (9, "Nancy", 50), (10, "Daniel", 78)]

# create dataframe from data and schema
df = spark.createDataFrame(data, schema) # createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)  


df.show()

df.printSchema()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1|  John| 19|
|  2| Smith| 29|
|  3|  Adam| 35|
|  4| Henry| 50|
|  5|  Mike| 78|
|  6|  Mary| 19|
|  7| Peter| 29|
|  8|  Cora| 35|
|  9| Nancy| 50|
| 10|Daniel| 78|
+---+------+---+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [4]:
df.schema # schema of the dataframe that can be used elsewhere

StructType([StructField('id', IntegerType(), True), StructField('name', StringType(), True), StructField('age', IntegerType(), True)])

In [5]:
# acess only specific column
df.select("name").show()

+------+
|  name|
+------+
|  John|
| Smith|
|  Adam|
| Henry|
|  Mike|
|  Mary|
| Peter|
|  Cora|
| Nancy|
|Daniel|
+------+



In [6]:
# access column with conditions
df.filter(df.age > 30).show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  3|  Adam| 35|
|  4| Henry| 50|
|  5|  Mike| 78|
|  8|  Cora| 35|
|  9| Nancy| 50|
| 10|Daniel| 78|
+---+------+---+



## End to end Example

In [7]:


flightData2015 = spark.read.csv("Data/2015-summary.csv", header=True, inferSchema=True) # use schema inference to create dataframe i.e sample some data and infer schema from it

flightData2015.show()
flightData2015.schema

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', IntegerType(), True)])

In [8]:
flightData2015.take(3) 

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [9]:
flightData2015.sort("count").explain() # explain() will show the execution plan of the query

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#57 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#57 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#85]
      +- FileScan csv [DEST_COUNTRY_NAME#55,ORIGIN_COUNTRY_NAME#56,count#57] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/amrit/Apache_Spark_Fuse/Data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [10]:
flightData2015.sort("count").count() # count() will return the number of rows in the dataframe

256

In [11]:
spark.conf.set("spark.sql.shuffle.partitions", "12")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [12]:
# dataframe to table view
flightData2015.createOrReplaceTempView("flight_data_2015")

In [13]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#55], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#55, 12), ENSURE_REQUIREMENTS, [id=#137]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#55], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#55] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/amrit/Apache_Spark_Fuse/Data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#55], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#55, 12), ENSURE_REQUIREMENTS, [id=#150]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#55], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#55] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)

In [14]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [15]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [16]:
# in Python we can use the same syntax as in SQL
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [17]:
# explan plan
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#167L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#55,destination_total#167L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#55], functions=[sum(count#57)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#55, 12), ENSURE_REQUIREMENTS, [id=#318]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#55], functions=[partial_sum(count#57)])
            +- FileScan csv [DEST_COUNTRY_NAME#55,count#57] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/amrit/Apache_Spark_Fuse/Data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [21]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"]) 


step4 = step3.selectExpr("sum(id)")
step4.collect() # 2500000000000

[Row(sum(id)=2500000000000)]

In [19]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#180L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#585]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#180L)])
            +- *(6) Project [id#180L]
               +- *(6) SortMergeJoin [id#180L], [id#174L], Inner
                  :- *(4) Sort [id#180L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#180L, 12), ENSURE_REQUIREMENTS, [id=#456]
                  :           +- *(3) Project [(id#172L * 5) AS id#180L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [id=#389]
                  :                    +- *(1) Range (2, 10000000, step=2, splits=6)
                

In [6]:
# accumulator 
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Practise").getOrCreate()
acc = spark.sparkContext.accumulator(0) # 0 is the initial value of the accumulator

def f(x):
    global acc
    acc += x

rdd = spark.sparkContext.parallelize([20, 30, 40, 50])
rdd.foreach(f)
final = acc.value
print(final)


22/10/19 12:23:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


140


In [8]:


# broadcast join 
from pyspark.sql.functions import broadcast
df1 = spark.createDataFrame([(1, "John Doe", 21)], ("id", "name", "age"))
df2 = spark.createDataFrame([(1, "USA")], ("id", "country"))
df1.join(broadcast(df2), "id").explain()



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [id#0L, name#1, age#2L, country#7]
   +- BroadcastHashJoin [id#0L], [id#6L], Inner, BuildRight, false
      :- Filter isnotnull(id#0L)
      :  +- Scan ExistingRDD[id#0L,name#1,age#2L]
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#24]
         +- Filter isnotnull(id#6L)
            +- Scan ExistingRDD[id#6L,country#7]




In [10]:
# udf  (user defined function)
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def toUpper(s):
    if s is not None:
        return s.upper()
toUpperUDF = udf(toUpper, StringType()) # register the function as a UDF and also specify the return type of the UDF
df1.select(toUpperUDF("name")).show()


# python udf is slower than scala udf because of the serialization overhead of python udf

+-------------+
|toUpper(name)|
+-------------+
|     JOHN DOE|
+-------------+



In [23]:
# pivot example 
df = spark.createDataFrame([(1,"USA",2015,100),(1,"USA",2016,200),(1,"USA",2017,300),(1,"USA",2018,400),(1,"USA",2019,500),(2,"India",2015,100),(2,"India",2016,200),(2,"India",2017,300),(2,"India",2018,400),(2,"India",2019,500),(3,"China",2015,100),(3,"China",2016,200),(3,"China",2017,300),(3,"China",2018,400),(3,"China",2019,500)],("id","country","year","population"))
df.show()

df.groupBy("year").pivot("country").sum("population").show()

+---+-------+----+----------+
| id|country|year|population|
+---+-------+----+----------+
|  1|    USA|2015|       100|
|  1|    USA|2016|       200|
|  1|    USA|2017|       300|
|  1|    USA|2018|       400|
|  1|    USA|2019|       500|
|  2|  India|2015|       100|
|  2|  India|2016|       200|
|  2|  India|2017|       300|
|  2|  India|2018|       400|
|  2|  India|2019|       500|
|  3|  China|2015|       100|
|  3|  China|2016|       200|
|  3|  China|2017|       300|
|  3|  China|2018|       400|
|  3|  China|2019|       500|
+---+-------+----+----------+

+----+-----+-----+---+
|year|China|India|USA|
+----+-----+-----+---+
|2016|  200|  200|200|
|2018|  400|  400|400|
|2017|  300|  300|300|
|2019|  500|  500|500|
|2015|  100|  100|100|
+----+-----+-----+---+



In [28]:
#window function
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg
windowSpec = Window.partitionBy("country").orderBy(col("population").desc())
df.withColumn("population_rank", avg("population").over(windowSpec)).show()



+---+-------+----+----------+---------------+
| id|country|year|population|population_rank|
+---+-------+----+----------+---------------+
|  3|  China|2019|       500|          500.0|
|  3|  China|2018|       400|          450.0|
|  3|  China|2017|       300|          400.0|
|  3|  China|2016|       200|          350.0|
|  3|  China|2015|       100|          300.0|
|  2|  India|2019|       500|          500.0|
|  2|  India|2018|       400|          450.0|
|  2|  India|2017|       300|          400.0|
|  2|  India|2016|       200|          350.0|
|  2|  India|2015|       100|          300.0|
|  1|    USA|2019|       500|          500.0|
|  1|    USA|2018|       400|          450.0|
|  1|    USA|2017|       300|          400.0|
|  1|    USA|2016|       200|          350.0|
|  1|    USA|2015|       100|          300.0|
+---+-------+----+----------+---------------+

